# Pointer Generator Model

In [29]:
import torch
from PICOHelper import get_pico_datasets
from NewsroomHelper import get_newsroom_datasets
from models import PointerGeneratorModel
from model_helpers import loss_function, error_function
from utils import get_index_words, produce_attention_visualization_file
from pytorch_helper import VariableBatchDataLoader, ModelManipulator, plot_learning_curves

## Parameters

In [2]:
# training parameters
BATCH_SIZE = 64
NUM_EPOCHS = 1
LEARNING_RATE = 1e-2
# INITIAL_ACCUMULATOR_VALUE = 0.1
GAMMA = 1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
BEAM_SIZE = 4

True


## Get Data

In [3]:
# pico_dataset_train, pico_dataset_dev, pico_dataset_test = get_pico_datasets()
newsroom_dataset_train, newsroom_dataset_dev, newsroom_dataset_test = get_newsroom_datasets(with_oov=True)
word_vectors = newsroom_dataset_train.word_vectors
start_index = newsroom_dataset_train.word_indices['<start>']
end_index = newsroom_dataset_train.word_indices['<end>']

11029 3676 3678
retrieving word2vec model from file


## Create Model

In [ ]:
pointer_generator_model = PointerGeneratorModel(word_vectors, start_index, end_index, num_hidden1=None, num_hidden2=None, with_coverage=False, gamma=GAMMA)

## Train and Save Model

In [ ]:
dataloader = VariableBatchDataLoader(newsroom_dataset_train, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.Adam(pointer_generator_model.parameters(),
                             lr=LEARNING_RATE)
# optimizer = torch.optim.Adagrad((pointer_generator_model.cuda() if USE_CUDA else pointer_generator_model).parameters(),
#                                 lr=LEARNING_RATE, initial_accumulator_value=INITIAL_ACCUMULATOR_VALUE)
model_manip = ModelManipulator(pointer_generator_model, optimizer, loss_function, error_function, use_cuda=USE_CUDA)
train_stats, val_stats = model_manip.train(dataloader, NUM_EPOCHS, dataset_val=newsroom_dataset_dev, stats_every=10, verbose_every=10)

In [ ]:
%debug

In [ ]:
torch.save(pointer_generator_model, 'models/pointer_generator_temp.model')

In [4]:
pointer_generator_model = torch.load('models/pointer_generator_temp.model')

/home/jered/Documents/Projects/Summarization/venv/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'models.PointerGeneratorModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


## Plot

In [ ]:
plot_learning_curves(training_values=train_stats, validation_values=val_stats, figure_name='graphs/pointer_generator_training_temp')

In [13]:
batch = newsroom_dataset_dev[2:3]
results, oov_indices = pointer_generator_model(batch['text'].cuda(), batch['text_length'].cuda(), batch['text_oov_indices'], beam_size=BEAM_SIZE)
# generated_output = generator_model(batch['text'], batch['text_length'])

In [32]:
loss, summary_info = results[0]
for i in range(len(summary_info[0])):
    summary_indices, summary_length = summary_info[0][i], summary_info[1][i]
    r_summary_indices, r_summary_length = batch['summary'][i].numpy(), batch['summary_length'][i].numpy()
    text_indices, text_length = batch['text'][i].numpy(), batch['text_length'][i].numpy()
    oov_words = {v:k for k,v in batch['text_oov_indices'][i].items()}
    
    text = get_index_words(text_indices[:text_length], newsroom_dataset_train.words, oov_words=oov_words)
    reference_summary = get_index_words(r_summary_indices[:r_summary_length], newsroom_dataset_train.words, oov_words=oov_words)
    decoded_summary = get_index_words(summary_indices[:summary_length], newsroom_dataset_train.words, oov_words=oov_words)
    print("text", text)
    print("reference summary", reference_summary)
    print("decoded summary", decoded_summary)
    print(loss[i])

text ['<start>', 'by', 'meredith', 'mandell', ',', 'special', 'for', 'usa', 'today', 'one', 'survivor', 'dies', 'in', 'israel', 'every', 'hour', ',', 'according', 'to', 'the', 'foundation', 'for', 'the', 'benefit', 'of', 'holocaust', 'survivors', 'in', 'israel', '.', 'president', 'mahmoud', 'ahmadinejad', 'claims', 'that', 'israel', 'is', 'bluffing', 'when', 'it', 'threatens', 'to', 'bomb', 'iran', "'s", 'nuclear', 'sites', 'to', 'stop', 'tehran', 'from', 'building', 'its', 'first', 'atomic', 'bomb', '.', 'iran', 'insists', 'its', 'nuclear', 'program', 'is', 'peaceful', ',', 'focused', 'on', 'generating', 'power', 'to', 'boost', 'its', 'economy', '.', 'sept', 'qqq', '(', 'reuters', ')', '-', 'israeli', 'president', 'shimon', 'peres', 'offered', 'his', 'iranian', 'counterpart', ',', 'mahmoud', 'ahmadinejad', ',', 'a', 'history', 'lesson', 'on', 'thursday', ',', 'saying', 'his', 'lack', 'of', 'knowledge', 'about', 'the', 'region', 'was', 'an', 'embarrassment', '.', 'the', 'outspoken', 'i

In [43]:
_, summary_info = results[0]
i = 0

summary_indices, summary_length = summary_info[0][i], summary_info[1][i]
attentions, p_gens = [[float(f) for f in vector] for vector in summary_info[3][i]], [float(f) for f in summary_info[4][i]]
r_summary_indices, r_summary_length = batch['summary'][i].numpy(), batch['summary_length'][i].numpy()
text_indices, text_length = batch['text'][i].numpy(), batch['text_length'][i].numpy()
oov_words = {v:k for k,v in batch['text_oov_indices'][i].items()}

text = get_index_words(text_indices[:text_length], newsroom_dataset_train.words, oov_words=oov_words)
reference_summary = get_index_words(r_summary_indices[:r_summary_length], newsroom_dataset_train.words, oov_words=oov_words)
decoded_summary = get_index_words(summary_indices[:summary_length], newsroom_dataset_train.words, oov_words=oov_words)
produce_attention_visualization_file('graphs/attn_vis_data.json', text, decoded_summary, reference_summary, attentions, p_gens)